In [1]:
import glob
import pandas as pd
import numpy as np
from pathlib import Path
from itertools import product
import json

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
N_REPETITIONS = 10
N_ITERATIONS = 25
FPS_VALUES = [10, 12, 14, 15, 17, 20]

In [3]:
base_path = "data/ronin/rs"
paths = {"20 EPIC": f"{base_path}/*/checkpoints/it*/*20*True.json",
         "18 EPIC": f"{base_path}/*/checkpoints/it*/*18*True.json",
         "17 EPIC": f"{base_path}/*/checkpoints/it*/*17*True.json",
         "15 EPIC": f"{base_path}/*/checkpoints/it*/*15*True.json",
         "14 EPIC": f"{base_path}/*/checkpoints/it*/*14*True.json",
         "12 EPIC": f"{base_path}/*/checkpoints/it*/*12*True.json",
         "10 EPIC": f"{base_path}/*/checkpoints/it*/*10*True.json",

         "20 LOW": f"{base_path}/*/checkpoints/it*/*20*False.json",
         "10 LOW": f"{base_path}/*/checkpoints/it*/*10*False.json",

         }

for name, pattern in paths.items():
    n = len(glob.glob(pattern))
    print(
        f"For {name} there are {n}/{N_REPETITIONS*N_ITERATIONS} ready {n//N_ITERATIONS}")

For 20 EPIC there are 250/250 ready 10
For 18 EPIC there are 250/250 ready 10
For 17 EPIC there are 250/250 ready 10
For 15 EPIC there are 250/250 ready 10
For 14 EPIC there are 250/250 ready 10
For 12 EPIC there are 250/250 ready 10
For 10 EPIC there are 250/250 ready 10
For 20 LOW there are 250/250 ready 10
For 10 LOW there are 250/250 ready 10


In [4]:
def process_df(df):
    record_df = pd.json_normalize(df['records'])
    # concatenate records data
    df = pd.concat([df, record_df],  axis=1)

    # drop original column
    df = df.drop(['records', 'index'], axis=1)

    # drop infractions columns

    # remove prefixes from column name
    df.columns = df.columns.str.removeprefix('meta.')
    df.columns = df.columns.str.removeprefix('scores.')
    df['driving_score'] = df['score_composed'] / 100
    # df = df.rename(columns={"index": "route_index"})
    df = df.set_index(['fps', 'highquality', 'rep', 'it', 'route_id'])
    return df

In [5]:

def make_df(path: str):

    # DATA FROM PATH
    data = {}
    data['path'] = path
    stem = Path(path).stem.split("_")
    data['fps'] = int(stem[1])
    data['highquality'] = stem[3]
    match path.split("/"):
        case ["data", "ronin", "rs", rep, "checkpoints", it, *objects]:
            data['rep'] = int(rep)
            data['it'] = int(it[2:])
    # print(data)

    # READ CHECKPOINT
    with open(path, "r") as f:
        content = json.load(f)

        data['records'] = content['_checkpoint']['records']
        if not data['records']:
            return

        df = pd.DataFrame(data)
        return process_df(df)


dfs = []
pattern = f"{base_path}/*/checkpoints/it*/*.json"
for file in glob.glob(pattern):
    dfs.append(make_df(file))

In [6]:
df = pd.concat(dfs).sort_index()
df

path                      status                      infractions.collisions_layout infractions.collisions_pedestrian                     infractions.collisions_vehicle                    infractions.outside_route_lanes                              infractions.red_light infractions.route_dev infractions.route_timeout infractions.stop_infraction                        infractions.vehicle_blocked  duration_game  duration_system  route_length  score_composed  score_penalty  score_route  driving_score
fps highquality rep it route_id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
10  False       0   0  RouteScenario_585  data/ronin/rs/0/checkpoints/it0/fps_10_highqua...                   Completed                                                 []                                []                                                 []                                                 []                                                 []                    []                        []                          []                                                 []      99.800001       190.234782    521.709640      100.000000       1.000000   100.000000       1.000000
                    1  RouteScenario_33   data/ronin/rs/0/checkpoints/it1/fps_10_highqua...  Failed - Agent got blocked  [Agent collided against object with type=stati...                                []                                                 []  [Agent went outside its route lanes for about ...  [Agent ran a red light 341 at (x=186.95, y=226...                    []                        []                          []  [Agent got blocked at (x=186.444, y=309.387, z...     208.700003       453.360121    292.742151       14.697936       0.432891    33.952991       0.146979
                    2  RouteScenario_439  data/ronin/rs/0/checkpoints/it2/fps_10_highqua...    Failed - Agent timed out  [Agent collided against object with type=stati...                                []  [Agent collided against object with type=vehic...  [Agent went outside its route lanes for about ...  [Agent ran a red light 662 at (x=264.0, y=-302...                    []          [Route timeout.]                          []                                                 []     176.100003       450.602438    209.466124       21.555446       0.221831    97.170735       0.215554
                    3  RouteScenario_494  data/ronin/rs/0/checkpoints/it3/fps_10_highqua...                   Completed                                                 []                                []  [Agent collided against object with type=vehic...                                                 []                                                 []                    []                        []                          []                                                 []     121.100002       285.478132    607.780443       60.000000       0.600000   100.000000       0.600000
                    4  RouteScenario_591  data/ronin/rs/0/checkpoints/it4/fps_10_highqua...                   Completed                                                 []                                []                                                 []  [Agent went outside its route lanes for about ...                                                 []                    []                        []                          []                                                 []     104.200002       204.950902    395.800026       98.036363 

In [7]:
# Get an index of minimal driving score for each fidelity value for each repetition
solutions = df.groupby(
    ['fps', 'highquality', 'rep'],)['driving_score'].idxmin()

# extract route id
solutions = solutions.apply(lambda x: x[-1].split("_")[-1])
solutions = solutions.rename("scenario_id")
solutions

fps  highquality  rep
10   False        0      142
                  1      476
                  2      233
                  3      150
                  4      228
                  5      533
                  6      492
                  7      140
                  8      491
                  9      296
     True         0      476
                  1       11
                  2       72
                  3      277
                  4      533
                  5      419
                  6      140
                  7      691
                  8      532
                  9      335
12   True         0      297
                  1      109
                  2      161
                  3      419
                  4      310
                  5      534
                  6      532
                  7      324
                  8      490
                  9      248
14   True         0      161
                  1      492
                  2      310
                  3  

In [8]:
solutions_dict = solutions.groupby(
    ['fps', 'highquality']).apply(list).apply(lambda x: ",".join(x))

solutions_dict

fps  highquality
10   False          142,476,233,150,228,533,492,140,491,296
     True             476,11,72,277,533,419,140,691,532,335
12   True           297,109,161,419,310,534,532,324,490,248
14   True           161,492,310,342,151,324,241,248,296,392
15   True           309,310,342,274,324,241,248,273,392,261
17   True           245,296,594,490,248,273,392,589,248,245
18   True           296,594,599,248,273,392,261,248,245,245
20   False          266,490,248,273,392,261,248,245,245,509
     True           490,248,273,392,261,248,260,231,509,248
Name: scenario_id, dtype: object

^ should be 90, 10 for each fidelity value

In [9]:
solutions_dict.to_csv("rs_solutions.csv")

In [10]:
df = pd.read_csv("rs_solutions.csv").set_index(['fps', 'highquality'])
df = df.xs(True, level='highquality')

for fps, row in df.iterrows():
    print(fps, row['scenario_id'])

10 476,11,72,277,533,419,140,691,532,335
12 297,109,161,419,310,534,532,324,490,248
14 161,492,310,342,151,324,241,248,296,392
15 309,310,342,274,324,241,248,273,392,261
17 245,296,594,490,248,273,392,589,248,245
18 296,594,599,248,273,392,261,248,245,245
20 490,248,273,392,261,248,260,231,509,248
